PO/ER460 - Projeto Final

SELEÇÃO E ORDENAMENTO DE ELEMENTOS DA MATRIZ SWOT ATRAVÉS DE UMA ABORDAGEM MULTICRITÉRIO E DE DECISÕES EM GRUPO

- Augusto Chebel Machado (231762)
- Émerson José Ferri
- Lucas Tramonte (182697)

# O que esse código faz?

- A partir do método PROMETHEE, realizamos o ordenamento para cada quadrante da matriz SWOT (Oportunidades, Ameaças, Pontos Fracos e Pontos Fortes);
- Cada decisor gera seu ordenamento;
- Após o ordenamento individual, através dos métodos de Borda e de Condorcet para decisões em grupo, gera-se os ordenamentos para o conjunto de decisores.

# Importando bibliotecas e acessando o Drive

In [ ]:
#Importando bibliotecas e acessando o Drive
import pandas as pd
import numpy as np

In [ ]:
!pip install gspread

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Código que importa dados do Google Sheets

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

from google.colab import files

# Definindo as funções para rodar o PROMETHEE


In [ ]:
# funções de preferência
def f_pref(df, criterio, tipo, func='usual', q=0, p=0):
  n_alt = len(df)
  m = np.zeros([n_alt, n_alt])
  for i in range(0, n_alt):
    for j in range(0, n_alt):
      if tipo == 'max':
        d = df[criterio][i] - df[criterio][j]
      elif tipo == 'min':
        d = df[criterio][j] - df[criterio][i]

      if func == 'usual':
        if d <= 0:
          m[i][j] = 0
        else:
          m[i][j] = 1

      if func == 'U':
        if d <= q:
          m[i][j] = 0
        else:
          m[i][j] = 1

      if func == 'níveis':
        if d <= 0:
          m[i][j] = 0
        elif q < d <= p:
          m[i][j] = 1/2
        else:
          m[i][j] = 1

      if func == 'linear':
        if d <= q:
          m[i][j] = 0
        elif q < d <= p:
          m[i][j] = (d-q)/(p-q)
        else:
          m[i][j] = 1

  return m

# cálculo dos fluxos
def fluxo_pos(M):
  fluxo_pos = []
  for i in range(0, len(M)):
    phi = sum(M[i])/(len(M)-1)
    fluxo_pos.append(phi)
  return np.array(fluxo_pos)

def fluxo_neg(M):
  fluxo_neg = []
  for j in range(0, len(M)):
    phi = sum(M[...,j])/(len(M)-1)
    fluxo_neg.append(phi)
  return np.array(fluxo_neg)

def fluxo_liq(fluxo_pos, fluxo_neg):
  fluxo_liq = fluxo_pos - fluxo_neg
  return fluxo_liq

# Importando e definindo as funções para o Método de Condorcet

In [ ]:
!pip install condorcet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import condorcet

In [ ]:
def voto(lista_df):
  votos = []
  for df in lista_df:
    voto_decisor = {}
    for i in range(0, len(df)):
      voto_decisor.update({df.index.values[i] : i+1})
    votos.append(voto_decisor)
  return votos

# Oportunidades

In [ ]:
# Importando os dados do problema (oportunidades)

oport_a1 = pd.DataFrame(gc.open('PO/ER460 - Oportunidades').get_worksheet(0).get_all_records())
oport_a2 = pd.DataFrame(gc.open('PO/ER460 - Oportunidades').get_worksheet(1).get_all_records())
oport_a3 = pd.DataFrame(gc.open('PO/ER460 - Oportunidades').get_worksheet(2).get_all_records())
oport_a4 = pd.DataFrame(gc.open('PO/ER460 - Oportunidades').get_worksheet(3).get_all_records())
# oport_a5 = pd.DataFrame(gc.open('PO/ER460 - Oportunidades').get_worksheet(4).get_all_records())

In [ ]:
# tornando as alternativas como índices
def indice(df):
  df = df.set_index('Alternativas/Critérios')
  df.index.names = [None]
  return df

oport_a1 = indice(oport_a1)
oport_a2 = indice(oport_a2)
oport_a3 = indice(oport_a3)
oport_a4 = indice(oport_a4)
# oport_a5 = indice(oport_a5)

In [ ]:
# criando uma escala numérica e passando para o df
def escala(df):
  dic = {'Muito Alto':5, 'Alto':4 , 'Médio':3, 'Baixo':2, 'Muito Baixo':1}
  df['Probabilidade'] = df['Probabilidade'].map(dic)
  df['Custo'] = df['Custo'].map(dic)
  df['Benefício'] = df['Benefício'].map(dic)
  df['Visão'] = df['Visão'].map(dic)
  return df

oport_a1 = escala(oport_a1)
oport_a2 = escala(oport_a2)
oport_a3 = escala(oport_a3)
oport_a4 = escala(oport_a4)
# oport_a5 = escala(oport_a5)

In [ ]:
# criando função para calcular a matriz de preferência M
def M_oport(df, w):
  m_fortes = f_pref(df, 'Pontos Fortes', 'linear', 'max', q=1, p=4)
  m_fracos = f_pref(df, 'Pontos Fracos', 'linear', 'min', q=1, p=4)
  m_prob = f_pref(df, 'Probabilidade', 'max')
  m_custo = f_pref(df, 'Custo', 'min')
  m_benef = f_pref(df, 'Benefício', 'max')
  m_visao = f_pref(df, 'Visão', 'max')

  matrizes = [m_fortes, m_fracos, m_prob, m_custo, m_benef, m_visao]

  M = 0
  for i in range(0, len(matrizes)):
    M = M + w[i]*matrizes[i]

  return M

# vetor de pesos
w_oport = [0.21, 0.09, 0.13, 0.11, 0.12, 0.34]

# cálculo das matrizes de preferência para cada decisor (oportunidades):
M_a1 = M_oport(oport_a1, w_oport)
M_a2 = M_oport(oport_a2, w_oport)
M_a3 = M_oport(oport_a3, w_oport)
M_a4 = M_oport(oport_a4, w_oport)
# M_a5 = M_oport(oport_a5, w_oport)


In [ ]:
# calculando o fluxo líquido
def fl_oport(M):
  fl = fluxo_liq(fluxo_pos(M), fluxo_neg(M))
  return fl

fl_a1 = fl_oport(M_a1)
fl_a2 = fl_oport(M_a2)
fl_a3 = fl_oport(M_a3)
fl_a4 = fl_oport(M_a4)
# fl_a5 = fl_oport(M_a5)

In [ ]:
# # função para ordenamento
# def ordenamento(df, fl):
#   # gerando um dicionário para ranking
#   rank = {}
#   for i in range(0, len(df)):
#     rank.update({df.index.values[i] : fl[i]})

#   # gerando a ordem (decrescente)
#   ordem = []
#   for i in sorted(rank, key=rank.get, reverse=True):
#     ordem.append(i)
#     # print(i, rank[i])

#   print('O ranking das oportunidades é:')
#   for i in range(0, len(ordem)):
#     print(f'{i+1}° - {ordem[i]}')

In [ ]:
# função para ordenamento
def ordenamento(decisor, df, fl):
  # gerando um dicionário para ranking
  rank = {}
  for i in range(0, len(df)):
    rank.update({df.index.values[i] : fl[i]})

  tabela = pd.DataFrame(pd.Series(rank), columns=['Fluxo Líquido'])

  tabela = tabela.sort_values(by='Fluxo Líquido', ascending=False)

  ord = []
  for i in range(0, len(df)):
    ord.append(i+1)

  tabela[f'Rank D{decisor}'] = ord

  return tabela

In [ ]:
print('Avaliador 1:')
print(ordenamento(1, oport_a1, fl_a1))
print(80*'-')
print('Avaliador 2:')
print(ordenamento(2, oport_a2, fl_a2))
print(80*'-')
print('Avaliador 3:')
print(ordenamento(3, oport_a3, fl_a3))
print(80*'-')
print('Avaliador 4:')
print(ordenamento(4, oport_a4, fl_a4))
# print(80*'-')
# print('Avaliador 5:')
# print(ordenamento(5, oport_a5, fl_a5))

Avaliador 1:
                                                    Fluxo Líquido  Rank D1
Crescimento do tema sustentabilidade                     0.420909        1
Recursos para projetos obras e ações                     0.339091        2
Uso da tecnologia BIM                                    0.111818        3
Melhoria do trabalho com uso das TIC                     0.052727        4
Eventos da Universidade                                  0.020909        5
Interesse de agentes externos para novas parcerias      -0.002727        6
Representação em órgãos externos                        -0.011818        7
Fundo patronos, nova via de financiamento das a...      -0.052727        8
Valorização e reconhecimento das competências i...      -0.117273        9
Inovação do ambiente universitário                      -0.167273       10
Lei de licitações 14.133 otimizando o processo ...      -0.198182       11
Amparo e segurança legal                                -0.395455       12
------------

In [ ]:
# gerando as tabelas para cada decisor (oportunidades)

ordem_oport_a1 = ordenamento(1, oport_a1, fl_a1)
ordem_oport_a2 = ordenamento(2, oport_a2, fl_a2)
ordem_oport_a3 = ordenamento(3, oport_a3, fl_a3)
ordem_oport_a4 = ordenamento(4, oport_a4, fl_a4)
# ordem_oport_a5 = ordenamento(5, oport_a5, fl_a5)

# criando um dataframe com os ordenamento de todos os decisores:
oportuni = pd.concat([ordem_oport_a1, ordem_oport_a2, ordem_oport_a3, ordem_oport_a4], axis=1)
oportuni
oportuni = oportuni.drop(['Fluxo Líquido'], axis=1)
oportuni

,Rank D1,Rank D2,Rank D3,Rank D4
Crescimento do tema sustentabilidade,1,1,1,2
Recursos para projetos obras e ações,2,6,2,6
Uso da tecnologia BIM,3,3,3,7
Melhoria do trabalho com uso das TIC,4,7,6,11
Eventos da Universidade,5,5,11,1
Interesse de agentes externos para novas parcerias,6,4,7,5
Representação em órgãos externos,7,10,12,12
"Fundo patronos, nova via de financiamento das ações da DEPI",8,9,9,8
Valorização e reconhecimento das competências independentemente das caraterísticas físicas,9,11,8,10
Inovação do ambiente universitário,10,2,5,3


In [ ]:
#@title Decisão em Grupo (Borda):
# criando uma escala para borda
borda_oport = {}
for i in range(0, len(oportuni)):
  borda_oport.update({i+1: (len(oportuni)-i)})

peso_dirigente = 1.2
borda_oport_dirig = {}
for i in range(0, len(oportuni)):
  borda_oport_dirig.update({i+1: (len(oportuni)-i)*peso_dirigente})

# substituindo os valores de ordenamento pelo valor da escala de Borda
# Ex: (1,2,3) vai virar (3,2,1)
for i in oportuni.columns:
  if i == oportuni.columns[0]: # se o decisor for a dirigente (peso maior)
    oportuni[i] = oportuni[i].map(borda_oport_dirig)
  else:
    oportuni[i] = oportuni[i].map(borda_oport)

# somando os valores de cada alternativa
oportuni['Valor'] = oportuni.sum(axis=1)
# oportuni

oport_final = pd.DataFrame(oportuni['Valor'])
oport_final = oport_final.sort_values(by='Valor', ascending=False)

ord = []
for i in range(0, len(oport_final)):
  ord.append(f'{i+1}°')

oport_final['Ordem Final (Borda)'] = ord
oport_final

,Valor,Ordem Final (Borda)
Crescimento do tema sustentabilidade,49.4,1°
Recursos para projetos obras e ações,38.2,2°
Uso da tecnologia BIM,38.0,3°
Inovação do ambiente universitário,32.6,4°
Eventos da Universidade,31.6,5°
Interesse de agentes externos para novas parcerias,31.4,6°
Melhoria do trabalho com uso das TIC,25.8,7°
Lei de licitações 14.133 otimizando o processo de licitações a partir de 2023,20.4,8°
"Fundo patronos, nova via de financiamento das ações da DEPI",19.0,9°
Valorização e reconhecimento das competências independentemente das caraterísticas físicas,14.8,10°


In [ ]:
#@title Decisão em grupo (Condorcet):

ordem_oport_a1 = ordenamento(1, oport_a1, fl_a1)
ordem_oport_a2 = ordenamento(2, oport_a2, fl_a2)
ordem_oport_a3 = ordenamento(3, oport_a3, fl_a3)
ordem_oport_a4 = ordenamento(4, oport_a4, fl_a4)

candidatos = ordem_oport_a1.index.values
candidatos = list(candidatos)

lista_oport = [ordem_oport_a1, ordem_oport_a2, ordem_oport_a3, ordem_oport_a4]
votos = voto(lista_oport)

# usando as funções da biblioteca condorcet para definir o ordenamento
evaluator = condorcet.CondorcetEvaluator(candidates=candidatos, votes=votos)
winners, rest_of_table = evaluator.get_n_winners(len(ordem_oport_a1))

oport_condorcet = pd.DataFrame(index=winners)

ord = []
for i in range(0, len(oport_condorcet)):
  ord.append(f'{i+1}°')
oport_condorcet['Ordem Final (Condorcet)'] = ord

oport_condorcet

,Ordem Final (Condorcet)
Crescimento do tema sustentabilidade,1°
Recursos para projetos obras e ações,2°
Uso da tecnologia BIM,3°
Eventos da Universidade,4°
Inovação do ambiente universitário,5°
Melhoria do trabalho com uso das TIC,6°
Interesse de agentes externos para novas parcerias,7°
"Fundo patronos, nova via de financiamento das ações da DEPI",8°
Lei de licitações 14.133 otimizando o processo de licitações a partir de 2023,9°
Representação em órgãos externos,10°


# Ameaças

In [ ]:
# Importando os dados do problema (ameaças)

ameaça_a1 = pd.DataFrame(gc.open('PO/ER460 - Ameaças').get_worksheet(0).get_all_records())
ameaça_a2 = pd.DataFrame(gc.open('PO/ER460 - Ameaças').get_worksheet(1).get_all_records())
ameaça_a3 = pd.DataFrame(gc.open('PO/ER460 - Ameaças').get_worksheet(2).get_all_records())
ameaça_a4 = pd.DataFrame(gc.open('PO/ER460 - Ameaças').get_worksheet(3).get_all_records())
# ameaça_a5 = pd.DataFrame(gc.open('PO/ER460 - Ameaças').get_worksheet(4).get_all_records())

In [ ]:
# tornando as alternativas como índices
def indice(df):
  df = df.set_index('Alternativas/Critérios')
  df.index.names = [None]
  return df

ameaça_a1 = indice(ameaça_a1)
ameaça_a2 = indice(ameaça_a2)
ameaça_a3 = indice(ameaça_a3)
ameaça_a4 = indice(ameaça_a4)
# ameaça_a5 = indice(ameaça_a5)

In [ ]:
# criando uma escala numérica e passando para o df
def escala(df):
  dic = {'Muito Alto':5, 'Alto':4 , 'Médio':3, 'Baixo':2, 'Muito Baixo':1}
  df['Probabilidade'] = df['Probabilidade'].map(dic)
  df['Custo'] = df['Custo'].map(dic)
  df['Impacto'] = df['Impacto'].map(dic)
  df['Visão'] = df['Visão'].map(dic)
  return df

ameaça_a1 = escala(ameaça_a1)
ameaça_a2 = escala(ameaça_a2)
ameaça_a3 = escala(ameaça_a3)
ameaça_a4 = escala(ameaça_a4)
# ameaça_a5 = escala(ameaça_a5)

In [ ]:
# criando função para calcular a matriz de preferência M
def M_ameaca(df, w):
  m_fracos = f_pref(df, 'Pontos Fracos', 'min', 'linear', q=1, p=4)
  m_fortes = f_pref(df, 'Pontos Fortes', 'max', 'linear', q=1, p=4)
  m_prob = f_pref(df, 'Probabilidade', 'max')
  m_custo = f_pref(df, 'Custo', 'min')
  m_impac = f_pref(df, 'Impacto', 'max')
  m_visao = f_pref(df, 'Visão', 'max')

  matrizes = [m_fortes, m_fracos, m_prob, m_custo, m_impac, m_visao]

  M = 0
  for i in range(0, len(matrizes)):
    M = M + w[i]*matrizes[i]

  return M

# vetor de pesos
w_ameaça = [0.18, 0.12, 0.18, 0.08, 0.12, 0.32]

# cálculo das matrizes de preferência para cada decisor (ameaças):
M_a1 = M_ameaca(ameaça_a1, w_ameaça)
M_a2 = M_ameaca(ameaça_a2, w_ameaça)
M_a3 = M_ameaca(ameaça_a3, w_ameaça)
M_a4 = M_ameaca(ameaça_a4, w_ameaça)
# M_a5 = M_ameaca(ameaça_a5, w_ameaça)

In [ ]:
# calculando o fluxo líquido
def fl_ameaca(M):
  fl = fluxo_liq(fluxo_pos(M), fluxo_neg(M))
  return fl

fl_a1 = fl_ameaca(M_a1)
fl_a2 = fl_ameaca(M_a2)
fl_a3 = fl_ameaca(M_a3)
fl_a4 = fl_ameaca(M_a4)
# fl_a5 = fl_ameaca(M_a5)

In [ ]:
# # função para ordenamento
# def ordenamento(df, fl):
#   # gerando um dicionário para ranking
#   rank = {}
#   for i in range(0, len(df)):
#     rank.update({df.index.values[i] : fl[i]})

#   # gerando a ordem (decrescente)
#   ordem = []
#   for i in sorted(rank, key=rank.get, reverse=True):
#     ordem.append(i)
#     # print(i, rank[i])

#   print('O ranking das oportunidades é:')
#   for i in range(0, len(ordem)):
#     print(f'{i+1}° - {ordem[i]}')

In [ ]:
# função para ordenamento
def ordenamento(decisor, df, fl):
  # gerando um dicionário para ranking
  rank = {}
  for i in range(0, len(df)):
    rank.update({df.index.values[i] : fl[i]})

  tabela = pd.DataFrame(pd.Series(rank), columns=['Fluxo Líquido'])

  tabela = tabela.sort_values(by='Fluxo Líquido', ascending=False)

  ord = []
  for i in range(0, len(df)):
    ord.append(i+1)

  tabela[f'Rank D{decisor}'] = ord

  return tabela

In [ ]:
print('Avaliador 1:')
print(ordenamento(1, ameaça_a1, fl_a1))
print(80*'-')
print('Avaliador 2:')
print(ordenamento(2, ameaça_a2, fl_a2))
print(80*'-')
print('Avaliador 3:')
print(ordenamento(3, ameaça_a3, fl_a3))
print(80*'-')
print('Avaliador 4:')
print(ordenamento(4, ameaça_a4, fl_a4))
# print(80*'-')
# print('Avaliador 5:')
# print(ordenamento(5, ameaça_a5, fl_a5))

Avaliador 1:
                                                    Fluxo Líquido  Rank D1
Resistência da comunidade universitária à polít...       0.509412        1
Crescimento da demanda além da capacidade opera...       0.445882        2
Falta de infraestrutura e resistência a automat...       0.351765        3
HIDS: o início da ocupação da fazenda argentina...       0.291765        4
Implicações orçamentárias                                0.241176        5
Baixa qualidade das empresas/ produtos/ serviço...       0.227059        6
Dificuldade de interface com outras áreas técni...       0.216471        7
Intrusão do novo governo do estado de São Paulo...       0.060000        8
Mudança no cenário político                              0.060000        9
Ameaças sanitárias                                       0.060000       10
Obstrução da procuradoria geral da universidade         -0.025882       11
Incertezas de critérios e volume de recursos pa...      -0.045882       12
Perda de tal

In [ ]:
# gerando as tabelas para cada decisor (ameaças)
ordem_ameaça_a1 = ordenamento(1, ameaça_a1, fl_a1)
ordem_ameaça_a2 = ordenamento(2, ameaça_a2, fl_a2)
ordem_ameaça_a3 = ordenamento(3, ameaça_a3, fl_a3)
ordem_ameaça_a4 = ordenamento(4, ameaça_a4, fl_a4)
# ordem_ameaça_a5 = ordenamento(5, ameaça_a5, fl_a5)

# criando um dataframe com os ordenamento de todos os decisores:
ameaça = pd.concat([ordem_ameaça_a1, ordem_ameaça_a2, ordem_ameaça_a3, ordem_ameaça_a4], axis=1)
ameaça = ameaça.drop(['Fluxo Líquido'], axis=1)
ameaça

,Rank D1,Rank D2,Rank D3,Rank D4
Resistência da comunidade universitária à políticas de planejamento e gestão integradas,1,1,3,1
Crescimento da demanda além da capacidade operacional,2,12,1,8
Falta de infraestrutura e resistência a automatização dos processos,3,2,16,14
HIDS: o início da ocupação da fazenda argentina trará alta demanda dos serviços da DEPI (obras e sustentabilidade),4,5,13,9
Implicações orçamentárias,5,10,7,13
Baixa qualidade das empresas/ produtos/ serviços contratados,6,6,11,5
Dificuldade de interface com outras áreas técnicas e administrativas da Unicamp,7,4,10,6
Intrusão do novo governo do estado de São Paulo na autonomia orçamentária da universidade,8,11,17,3
Mudança no cenário político,9,8,14,7
Ameaças sanitárias,10,15,12,4


In [ ]:
#@title Decisão em Grupo (Borda):
# criando uma escala para borda
borda_ameaça = {}
for i in range(0, len(ameaça)):
  borda_ameaça.update({i+1: (len(ameaça)-i)})

peso_dirigente = 1.2
borda_ameaça_dirig = {}
for i in range(0, len(ameaça)):
  borda_ameaça_dirig.update({i+1: (len(ameaça)-i)*peso_dirigente})

# substituindo os valores de ordenamento pelo valor da escala de Borda
# Ex: (1,2,3) vai virar (3,2,1)
for i in ameaça.columns:
  if i == ameaça.columns[0]: # se o decisor for a dirigente (peso maior)
    ameaça[i] = ameaça[i].map(borda_ameaça_dirig)
  else:
    ameaça[i] = ameaça[i].map(borda_ameaça)

# somando os valores de cada alternativa
ameaça['Valor'] = ameaça.sum(axis=1)
# ameaça

ameaça_final = pd.DataFrame(ameaça['Valor'])
ameaça_final = ameaça_final.sort_values(by='Valor', ascending=False)

ord = []
for i in range(0, len(ameaça_final)):
  ord.append(f'{i+1}°')

ameaça_final['Ordem Final (Borda)'] = ord
ameaça_final

,Valor,Ordem Final (Borda)
Resistência da comunidade universitária à políticas de planejamento e gestão integradas,73.6,1°
Crescimento da demanda além da capacidade operacional,56.4,2°
Dificuldade de interface com outras áreas técnicas e administrativas da Unicamp,51.4,3°
Baixa qualidade das empresas/ produtos/ serviços contratados,50.6,4°
Perda de talentos,49.2,5°
HIDS: o início da ocupação da fazenda argentina trará alta demanda dos serviços da DEPI (obras e sustentabilidade),48.0,6°
Falta de infraestrutura e resistência a automatização dos processos,44.2,7°
Implicações orçamentárias,43.8,8°
Mudança no cenário político,40.0,9°
Intrusão do novo governo do estado de São Paulo na autonomia orçamentária da universidade,39.2,10°


In [ ]:
#@title Decisão em Grupo (Condorcet):

ordem_ameaça_a1 = ordenamento(1, ameaça_a1, fl_a1)
ordem_ameaça_a2 = ordenamento(2, ameaça_a2, fl_a2)
ordem_ameaça_a3 = ordenamento(3, ameaça_a3, fl_a3)
ordem_ameaça_a4 = ordenamento(4, ameaça_a4, fl_a4)

candidatos = ordem_ameaça_a1.index.values
candidatos = list(candidatos)

lista_ameaça = [ordem_ameaça_a1, ordem_ameaça_a2, ordem_ameaça_a3, ordem_ameaça_a4]
votos = voto(lista_ameaça)

# usando as funções da biblioteca condorcet para definir o ordenamento
evaluator = condorcet.CondorcetEvaluator(candidates=candidatos, votes=votos)
winners, rest_of_table = evaluator.get_n_winners(len(ordem_ameaça_a1))

ameaça_condorcet = pd.DataFrame(index=winners)

ord = []
for i in range(0, len(ameaça_condorcet)):
  ord.append(f'{i+1}°')
ameaça_condorcet['Ordem Final (Condorcet)'] = ord

ameaça_condorcet

,Ordem Final (Condorcet)
Resistência da comunidade universitária à políticas de planejamento e gestão integradas,1°
Crescimento da demanda além da capacidade operacional,2°
Falta de infraestrutura e resistência a automatização dos processos,3°
Baixa qualidade das empresas/ produtos/ serviços contratados,4°
Dificuldade de interface com outras áreas técnicas e administrativas da Unicamp,5°
HIDS: o início da ocupação da fazenda argentina trará alta demanda dos serviços da DEPI (obras e sustentabilidade),6°
Mudança no cenário político,7°
Perda de talentos,8°


# Pontos Fracos

In [ ]:
# Importando os dados do problema (pontos fracos)

fraco_a1 = pd.DataFrame(gc.open('PO/ER460 - Pontos Fracos').get_worksheet(0).get_all_records())
fraco_a2 = pd.DataFrame(gc.open('PO/ER460 - Pontos Fracos').get_worksheet(1).get_all_records())
fraco_a3 = pd.DataFrame(gc.open('PO/ER460 - Pontos Fracos').get_worksheet(2).get_all_records())
fraco_a4 = pd.DataFrame(gc.open('PO/ER460 - Pontos Fracos').get_worksheet(3).get_all_records())
# fraco_a5 = pd.DataFrame(gc.open('PO/ER460 - Pontos Fracos').get_worksheet(4).get_all_records())

In [ ]:
# tornando as alternativas como índices
def indice(df):
  df = df.set_index('Alternativas/Critérios')
  df.index.names = [None]
  return df

fraco_a1 = indice(fraco_a1)
fraco_a2 = indice(fraco_a2)
fraco_a3 = indice(fraco_a3)
fraco_a4 = indice(fraco_a4)
# fraco_a5 = indice(fraco_a5)

In [ ]:
# criando uma escala numérica e passando para o df
def escala(df):
  dic = {'Muito Alto':5, 'Alto':4 , 'Médio':3, 'Baixo':2, 'Muito Baixo':1}
  df['Custo'] = df['Custo'].map(dic)
  df['Retorno'] = df['Retorno'].map(dic)
  df['Missão'] = df['Missão'].map(dic)
  df['Visão'] = df['Visão'].map(dic)
  return df

fraco_a1 = escala(fraco_a1)
fraco_a2 = escala(fraco_a2)
fraco_a3 = escala(fraco_a3)
fraco_a4 = escala(fraco_a4)
# fraco_a5 = escala(fraco_a5)

In [ ]:
# criando função para calcular a matriz de preferência M
def M_fraco(df, w):
  m_pot_ame = f_pref(df, 'Pot. Ameaças', 'max', 'linear', q=1, p=4)
  m_dif_oport = f_pref(df, 'Dif. Oport.', 'max', 'linear', q=1, p=4)
  m_custo = f_pref(df, 'Custo', 'min')
  m_ret = f_pref(df, 'Retorno', 'max')
  m_missao = f_pref(df, 'Missão', 'max')
  m_visao = f_pref(df, 'Visão', 'max')

  matrizes = [m_pot_ame, m_dif_oport, m_custo, m_ret, m_missao, m_visao]

  M = 0
  for i in range(0, len(matrizes)):
    M = M + w[i]*matrizes[i]

  return M

# vetor de pesos
w_fraco = [0.09, 0.06, 0.12, 0.10, 0.23, 0.40]

# cálculo das matrizes de preferência para cada decisor (pontos fracos):
M_a1 = M_fraco(fraco_a1, w_fraco)
M_a2 = M_fraco(fraco_a2, w_fraco)
M_a3 = M_fraco(fraco_a3, w_fraco)
M_a4 = M_fraco(fraco_a4, w_fraco)
# M_a5 = M_fraco(fraco_a5, w_fraco)

In [ ]:
# calculando o fluxo líquido
def fl_fraco(M):
  fl = fluxo_liq(fluxo_pos(M), fluxo_neg(M))
  return fl

fl_a1 = fl_fraco(M_a1)
fl_a2 = fl_fraco(M_a2)
fl_a3 = fl_fraco(M_a3)
fl_a4 = fl_fraco(M_a4)
# fl_a5 = fl_fraco(M_a5)

In [ ]:
# # função para ordenamento
# def ordenamento(df, fl):
#   # gerando um dicionário para ranking
#   rank = {}
#   for i in range(0, len(df)):
#     rank.update({df.index.values[i] : fl[i]})

#   # gerando a ordem (decrescente)
#   ordem = []
#   for i in sorted(rank, key=rank.get, reverse=True):
#     ordem.append(i)
#     # print(i, rank[i])

#   print('O ranking das oportunidades é:')
#   for i in range(0, len(ordem)):
#     print(f'{i+1}° - {ordem[i]}')

In [ ]:
# função para ordenamento
def ordenamento(decisor, df, fl):
  # gerando um dicionário para ranking
  rank = {}
  for i in range(0, len(df)):
    rank.update({df.index.values[i] : fl[i]})

  tabela = pd.DataFrame(pd.Series(rank), columns=['Fluxo Líquido'])

  tabela = tabela.sort_values(by='Fluxo Líquido', ascending=False)

  ord = []
  for i in range(0, len(df)):
    ord.append(i+1)

  tabela[f'Rank D{decisor}'] = ord

  return tabela

In [ ]:
print('Avaliador 1:')
print(ordenamento(1, fraco_a1, fl_a1))
print(80*'-')
print('Avaliador 2:')
print(ordenamento(2, fraco_a2, fl_a2))
print(80*'-')
print('Avaliador 3:')
print(ordenamento(3, fraco_a3, fl_a3))
print(80*'-')
print('Avaliador 4:')
print(ordenamento(4, fraco_a4, fl_a4))
# print(80*'-')
# print('Avaliador 5:')
# print(ordenamento(5, fraco_a5, fl_a5))

Avaliador 1:
                                                    Fluxo Líquido  Rank D1
Falta de clareza dos objetivos a serem alcançados        0.767500        1
Processos de trabalhos pouco definidos                   0.761667        2
Falta de sistema de informação adequado ao proc...       0.499167        3
Comunicação institucional deficiente                     0.391667        4
Falta de integração entre os setores da DEPI             0.056667        5
Falta de pessoal                                        -0.016667        6
Oferecer poucos treinamentos na universidade da...      -0.021667        7
Falha de comunicação interna entre as equipes           -0.031667        8
Falta de conhecimento do usuário sobre os servi...      -0.241667        9
Indefinição sobre a área de projetos de empreen...      -0.255833       10
Talentos pouco explorados                               -0.460000       11
Demora no retorno de algumas solicitações               -0.634167       12
Separação fí

In [ ]:
# gerando as tabelas para cada decisor (pontos fracos)
ordem_fraco_a1 = ordenamento(1, fraco_a1, fl_a1)
ordem_fraco_a2 = ordenamento(2, fraco_a2, fl_a2)
ordem_fraco_a3 = ordenamento(3, fraco_a3, fl_a3)
ordem_fraco_a4 = ordenamento(4, fraco_a4, fl_a4)
# ordem_fraco_a5 = ordenamento(5, fraco_a5, fl_a5)

# criando um dataframe com os ordenamento de todos os decisores:
pont_fracos = pd.concat([ordem_fraco_a1, ordem_fraco_a2, ordem_fraco_a3, ordem_fraco_a4], axis=1)
pont_fracos = pont_fracos.drop(['Fluxo Líquido'], axis=1)
pont_fracos

,Rank D1,Rank D2,Rank D3,Rank D4
Falta de clareza dos objetivos a serem alcançados,1,3,1,7
Processos de trabalhos pouco definidos,2,1,2,1
Falta de sistema de informação adequado ao processo atual,3,2,12,3
Comunicação institucional deficiente,4,7,9,6
Falta de integração entre os setores da DEPI,5,4,4,11
Falta de pessoal,6,12,10,8
Oferecer poucos treinamentos na universidade dado o conhecimento que possui,7,9,11,2
Falha de comunicação interna entre as equipes,8,10,7,10
Falta de conhecimento do usuário sobre os serviços prestados,9,11,3,5
Indefinição sobre a área de projetos de empreendimentos,10,5,5,4


In [ ]:
#@title Decisão em Grupo (Borda):
# criando uma escala para borda
borda_fraco = {}
for i in range(0, len(pont_fracos)):
  borda_fraco.update({i+1: (len(pont_fracos)-i)})

peso_dirigente = 1.2
borda_fraco_dirig = {}
for i in range(0, len(pont_fracos)):
  borda_fraco_dirig.update({i+1: (len(pont_fracos)-i)*peso_dirigente})

# substituindo os valores de ordenamento pelo valor da escala de Borda
# Ex: (1,2,3) vai virar (3,2,1)
for i in pont_fracos.columns:
  if i == pont_fracos.columns[0]: # se o decisor for a dirigente (peso maior)
    pont_fracos[i] = pont_fracos[i].map(borda_fraco_dirig)
  else:
    pont_fracos[i] = pont_fracos[i].map(borda_fraco)

# somando os valores de cada alternativa
pont_fracos['Valor'] = pont_fracos.sum(axis=1)
# pont_fracos

pont_fracos_final = pd.DataFrame(pont_fracos['Valor'])
pont_fracos_final = pont_fracos_final.sort_values(by='Valor', ascending=False)

ord = []
for i in range(0, len(pont_fracos_final)):
  ord.append(f'{i+1}°')

pont_fracos_final['Ordem Final (Borda)'] = ord
pont_fracos_final

,Valor,Ordem Final (Borda)
Processos de trabalhos pouco definidos,52.4,1°
Falta de clareza dos objetivos a serem alcançados,46.6,2°
Falta de sistema de informação adequado ao processo atual,38.2,3°
Falta de integração entre os setores da DEPI,33.8,4°
Indefinição sobre a área de projetos de empreendimentos,32.8,5°
Comunicação institucional deficiente,32.0,6°
Falta de conhecimento do usuário sobre os serviços prestados,29.0,7°
Oferecer poucos treinamentos na universidade dado o conhecimento que possui,28.4,8°
Talentos pouco explorados,22.6,9°
Falha de comunicação interna entre as equipes,22.2,10°


In [ ]:
#@title Decisão em Grupo (Condorcet):

ordem_fraco_a1 = ordenamento(1, fraco_a1, fl_a1)
ordem_fraco_a2 = ordenamento(2, fraco_a2, fl_a2)
ordem_fraco_a3 = ordenamento(3, fraco_a3, fl_a3)
ordem_fraco_a4 = ordenamento(4, fraco_a4, fl_a4)

candidatos = ordem_fraco_a1.index.values
candidatos = list(candidatos)

lista_fraco = [ordem_fraco_a1, ordem_fraco_a2, ordem_fraco_a3, ordem_fraco_a4]
votos = voto(lista_fraco)

# usando as funções da biblioteca condorcet para definir o ordenamento
evaluator = condorcet.CondorcetEvaluator(candidates=candidatos, votes=votos)
winners, rest_of_table = evaluator.get_n_winners(len(ordem_ameaça_a1))

fraco_condorcet = pd.DataFrame(index=winners)

ord = []
for i in range(0, len(fraco_condorcet)):
  ord.append(f'{i+1}°')
fraco_condorcet['Ordem Final (Condorcet)'] = ord

fraco_condorcet

,Ordem Final (Condorcet)
Falta de clareza dos objetivos a serem alcançados,1°
Processos de trabalhos pouco definidos,2°
Falta de sistema de informação adequado ao processo atual,3°
Falta de integração entre os setores da DEPI,4°
Indefinição sobre a área de projetos de empreendimentos,5°
Comunicação institucional deficiente,6°
Oferecer poucos treinamentos na universidade dado o conhecimento que possui,7°
Falta de conhecimento do usuário sobre os serviços prestados,8°
Falta de pessoal,9°
Talentos pouco explorados,10°


# Pontos Fortes

In [ ]:
# Importando os dados do problema (pontos fortes)

forte_a1 = pd.DataFrame(gc.open('PO/ER460 - Pontos Fortes').get_worksheet(0).get_all_records())
forte_a2 = pd.DataFrame(gc.open('PO/ER460 - Pontos Fortes').get_worksheet(1).get_all_records())
forte_a3 = pd.DataFrame(gc.open('PO/ER460 - Pontos Fortes').get_worksheet(2).get_all_records())
forte_a4 = pd.DataFrame(gc.open('PO/ER460 - Pontos Fortes').get_worksheet(3).get_all_records())
# forte_a5 = pd.DataFrame(gc.open('PO/ER460 - Pontos Fortes').get_worksheet(4).get_all_records())

In [ ]:
# tornando as alternativas como índices
def indice(df):
  df = df.set_index('Alternativas/Critérios')
  df.index.names = [None]
  return df

forte_a1 = indice(forte_a1)
forte_a2 = indice(forte_a2)
forte_a3 = indice(forte_a3)
forte_a4 = indice(forte_a4)
# forte_a5 = indice(forte_a5)

In [ ]:
# criando uma escala numérica e passando para o df
def escala(df):
  dic = {'Muito Alto':5, 'Alto':4 , 'Médio':3, 'Baixo':2, 'Muito Baixo':1}
  df['Missão'] = df['Missão'].map(dic)
  df['Visão'] = df['Visão'].map(dic)
  return df

forte_a1 = escala(forte_a1)
forte_a2 = escala(forte_a2)
forte_a3 = escala(forte_a3)
forte_a4 = escala(forte_a4)
# forte_a5 = escala(forte_a5)

In [ ]:
# criando função para calcular a matriz de preferência M
def M_forte(df, w):
  m_pot_oport = f_pref(df, 'Pot. Oport', 'max', 'linear', q=1, p=4)
  m_neut_ame = f_pref(df, 'Neut. Ameaça', 'max', 'linear', q=1, p=4)
  m_missao = f_pref(df, 'Missão', 'max')
  m_visao = f_pref(df, 'Visão', 'max')

  matrizes = [m_pot_oport, m_neut_ame, m_missao, m_visao]

  M = 0
  for i in range(0, len(matrizes)):
    M = M + w[i]*matrizes[i]

  return M

# vetor de pesos
w_forte = [0.10, 0.17, 0.25, 0.48]

# cálculo das matrizes de preferência para cada decisor (pontos fortes):
M_a1 = M_forte(forte_a1, w_forte)
M_a2 = M_forte(forte_a2, w_forte)
M_a3 = M_forte(forte_a3, w_forte)
M_a4 = M_forte(forte_a4, w_forte)
# M_a5 = M_forte(forte_a5, w_forte)

In [ ]:
# calculando o fluxo líquido
def fl_forte(M):
  fl = fluxo_liq(fluxo_pos(M), fluxo_neg(M))
  return fl

fl_a1 = fl_forte(M_a1)
fl_a2 = fl_forte(M_a2)
fl_a3 = fl_forte(M_a3)
fl_a4 = fl_forte(M_a4)
# fl_a5 = fl_forte(M_a5)

In [ ]:
# # função para ordenamento
# def ordenamento(df, fl):
#   # gerando um dicionário para ranking
#   rank = {}
#   for i in range(0, len(df)):
#     rank.update({df.index.values[i] : fl[i]})

#   # gerando a ordem (decrescente)
#   ordem = []
#   for i in sorted(rank, key=rank.get, reverse=True):
#     ordem.append(i)
#     # print(i, rank[i])

#   print('O ranking das oportunidades é:')
#   for i in range(0, len(ordem)):
#     print(f'{i+1}° - {ordem[i]}')

In [ ]:
# função para ordenamento
def ordenamento(decisor, df, fl):
  # gerando um dicionário para ranking
  rank = {}
  for i in range(0, len(df)):
    rank.update({df.index.values[i] : fl[i]})

  tabela = pd.DataFrame(pd.Series(rank), columns=['Fluxo Líquido'])

  tabela = tabela.sort_values(by='Fluxo Líquido', ascending=False)

  ord = []
  for i in range(0, len(df)):
    ord.append(i+1)

  tabela[f'Rank D{decisor}'] = ord

  return tabela

In [ ]:
print('Avaliador 1:')
print(ordenamento(1, forte_a1, fl_a1))
print(80*'-')
print('Avaliador 2:')
print(ordenamento(2, forte_a2, fl_a2))
print(80*'-')
print('Avaliador 3:')
print(ordenamento(3, forte_a3, fl_a3))
print(80*'-')
print('Avaliador 4:')
print(ordenamento(4, forte_a4, fl_a4))
# print(80*'-')
# print('Avaliador 5:')
# print(ordenamento(5, forte_a5, fl_a5))

Avaliador 1:
                                                    Fluxo Líquido  Rank D1
Profissionais engajados                                  0.643556        1
Qualidade técnica dos profissionais                      0.624667        2
Incentivo à novas iniciativas                            0.529333        3
Visão sistemica                                          0.506667        4
Conhecimento abrangente e disseminável                   0.491111        5
Papel atuante como suporte à tomada de decisões...       0.491111        6
Ações geram impacto na Comunidade                        0.407556        7
Autonomia e confiança na tomada de decisões             -0.097333        8
Clareza e detalhamento dos procedimentos e proc...      -0.340889        9
Comunicação interna                                     -0.351111       10
Coerência na composição da DEPI frente a estrut...      -0.402222       11
Incetivo à capacitação                                  -0.439111       12
Equipe e amb

In [ ]:
# gerando as tabelas para cada decisor (pontos fortes)
ordem_forte_a1 = ordenamento(1, forte_a1, fl_a1)
ordem_forte_a2 = ordenamento(2, forte_a2, fl_a2)
ordem_forte_a3 = ordenamento(3, forte_a3, fl_a3)
ordem_forte_a4 = ordenamento(4, forte_a4, fl_a4)
# ordem_forte_a5 = ordenamento(5, forte_a5, fl_a5)

# criando um dataframe com os ordenamento de todos os decisores:
pont_fortes = pd.concat([ordem_forte_a1, ordem_forte_a2, ordem_forte_a3, ordem_forte_a4], axis=1)
pont_fortes = pont_fortes.drop(['Fluxo Líquido'], axis=1)
pont_fortes

,Rank D1,Rank D2,Rank D3,Rank D4
Profissionais engajados,1,11,1,2
Qualidade técnica dos profissionais,2,1,2,1
Incentivo à novas iniciativas,3,6,10,8
Visão sistemica,4,2,3,6
Conhecimento abrangente e disseminável,5,8,16,3
Papel atuante como suporte à tomada de decisões na universidade,6,4,8,11
Ações geram impacto na Comunidade,7,9,11,4
Autonomia e confiança na tomada de decisões,8,3,15,13
Clareza e detalhamento dos procedimentos e processos padronizados,9,12,12,10
Comunicação interna,10,14,4,14


In [ ]:
#@title Decisão em Grupo (Borda):
# criando uma escala para borda
borda_forte = {}
for i in range(0, len(pont_fortes)):
  borda_forte.update({i+1: (len(pont_fortes)-i)})

peso_dirigente = 1.2
borda_forte_dirig = {}
for i in range(0, len(pont_fortes)):
  borda_forte_dirig.update({i+1: (len(pont_fortes)-i)*peso_dirigente})

# substituindo os valores de ordenamento pelo valor da escala de Borda
# Ex: (1,2,3) vai virar (3,2,1)
for i in pont_fortes.columns:
  if i == pont_fortes.columns[0]: # se o decisor for a dirigente (peso maior)
    pont_fortes[i] = pont_fortes[i].map(borda_forte_dirig)
  else:
    pont_fortes[i] = pont_fortes[i].map(borda_forte)

# somando os valores de cada alternativa
pont_fortes['Valor'] = pont_fortes.sum(axis=1)
# pont_fortes

pont_fortes_final = pd.DataFrame(pont_fortes['Valor'])
pont_fortes_final = pont_fortes_final.sort_values(by='Valor', ascending=False)

ord = []
for i in range(0, len(pont_fortes_final)):
  ord.append(f'{i+1}°')

pont_fortes_final['Ordem Final (Borda)'] = ord
pont_fortes_final

,Valor,Ordem Final (Borda)
Qualidade técnica dos profissionais,65.0,1°
Profissionais engajados,56.2,2°
Visão sistemica,55.6,3°
Incentivo à novas iniciativas,43.8,4°
Papel atuante como suporte à tomada de decisões na universidade,41.2,5°
Incetivo à capacitação,41.0,6°
Ações geram impacto na Comunidade,39.0,7°
Conhecimento abrangente e disseminável,38.4,8°
Autonomia e confiança na tomada de decisões,30.8,9°
Coerência na composição da DEPI frente a estrutura organizacional da Unicamp,30.2,10°


In [ ]:
#@title Decisão em Grupo (Condorcet):

ordem_forte_a1 = ordenamento(1, forte_a1, fl_a1)
ordem_forte_a2 = ordenamento(2, forte_a2, fl_a2)
ordem_forte_a3 = ordenamento(3, forte_a3, fl_a3)
ordem_forte_a4 = ordenamento(4, forte_a4, fl_a4)

candidatos = ordem_forte_a1.index.values
candidatos = list(candidatos)

lista_forte = [ordem_forte_a1, ordem_forte_a2, ordem_forte_a3, ordem_forte_a4]
votos = voto(lista_forte)

# usando as funções da biblioteca condorcet para definir o ordenamento
evaluator = condorcet.CondorcetEvaluator(candidates=candidatos, votes=votos)
winners, rest_of_table = evaluator.get_n_winners(len(ordem_forte_a1))

forte_condorcet = pd.DataFrame(index=winners)

ord = []
for i in range(0, len(forte_condorcet)):
  ord.append(f'{i+1}°')
forte_condorcet['Ordem Final (Condorcet)'] = ord

forte_condorcet

,Ordem Final (Condorcet)
Profissionais engajados,1°
Qualidade técnica dos profissionais,2°
Visão sistemica,3°
Papel atuante como suporte à tomada de decisões na universidade,4°
Incetivo à capacitação,5°
Incentivo à novas iniciativas,6°
Conhecimento abrangente e disseminável,7°
Ações geram impacto na Comunidade,8°
Autonomia e confiança na tomada de decisões,9°
Clareza e detalhamento dos procedimentos e processos padronizados,10°
